In [81]:
# import basic libraries and report versions
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import platform
print("python version: ",platform.python_version())
print("pandas version: ",pd.__version__)
print("numpy version: ",np.__version__)
print("seaborn version: ",sns.__version__)
print("pandas-profiling version: ",pdp.__version__)
print("matplotlib version: ",matplotlib.__version__)
sns.set_style('whitegrid')

python version:  3.7.4
pandas version:  0.25.1
numpy version:  1.17.2
seaborn version:  0.9.0
pandas-profiling version:  2.4.0
matplotlib version:  3.1.1


In [82]:
from unidecode import unidecode

In [83]:
pip install geonamescache

Note: you may need to restart the kernel to use updated packages.


In [84]:

source = open('headlines.txt', 'r')
headlines = []

for line in source.readlines():
    headlines.append(unidecode(line).strip().replace('?','')) 
    
  

In [85]:
import geonamescache

gc = geonamescache.GeonamesCache()
gc_countries = gc.get_countries()
gc_cities = gc.get_cities()


In [86]:
cityNames = []
for instance in gc_cities:
   cityNames.append (gc_cities[instance]['name'])
   

countryNames = []
for instance in gc_countries:
   countryNames.append (gc_countries[instance]['name'])
   


In [87]:
wellknown={'London':'United Kingdom', 'Rotterdam': 'Netherlands', 'Florence': 'Italy'}

def chooseName(past, suggested):    
    if past == '':
        return suggested
    elif past in suggested:
        return suggested
    else:
        return past
    
def getCountry(pastName, suggestedName):  
    name = chooseName(pastName, suggestedName)
    code = gc.get_countries_by_names()[name]['iso']
    return [name, code]

capitals = {}

df = pd.DataFrame(list(zip(headlines)), columns =['headline'])
for col in ['countries', 'cities', 'country_codes', 'lat', 'long']:
    df[col] = ''

In [88]:
import re
searchme = r".*\b{}\b.*"

def findCity(city, line):
    regexp = re.compile(searchme.format(city))    
    result = regexp.search(line)    
    return (result != None)
    
# For cities whose name is used in more than one country:
  # Country capitals are more likely than not.
  # A few cities are handled with the 'wellknown' list
  # In all other cases, the country whose code is alphabetically last prevails. This favors the US, which makses sense given 
    # the source material.
for ind in df.index:
    line = df['headline'][ind]
    countryNotInLine = True
    
    for country in countryNames:
        if country in line: 
            [df['countries'][ind], df['country_codes'][ind]] = getCountry('',country)
            countryNotInLine = False
    for city in cityNames:
        cityFound = ''
        city = city.strip()
        
        if city in line and findCity(city, line):           
            cityFound = chooseName(cityFound,city)
            df['cities'][ind] = cityFound
            if countryNotInLine:
             if cityFound in wellknown:
               [df['countries'][ind], df['country_codes'][ind]] = getCountry('', wellknown[cityFound])
             elif cityFound in capitals:
                [df['countries'][ind], df['country_codes'][ind]] = getCountry('', capitals[cityFound])
             else:
                find_cities = gc.get_cities_by_name(cityFound)               
                codes = []
                for cit in find_cities:                    
                    for key in cit:
                        ccd = cit[key]['countrycode']                       
                        codes.append(ccd)
                for cd in codes:                    
                    df['countries'][ind] = gc_countries[cd]['name']
                    df['country_codes'][ind] = cd
                    if city == gc_countries[cd]['capital'].strip():                          
                        capitals[city] = df['countries'][ind]
                    
                        
                        
df['country_codes'].replace('', np.nan, inplace=True)
df.dropna(inplace=True)            
df.head(15)

,headline,countries,cities,country_codes,lat,long
0,Zika Outbreak Hits Miami,United States,Miami,US,,
1,Could Zika Reach New York City,United States,New York City,US,,
2,First Case of Zika in Miami Beach,United States,Miami Beach,US,,
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife,BR,,
4,Dallas man comes down with case of Zika,United States,Dallas,US,,
5,Trinidad confirms first Zika case,Uruguay,Trinidad,UY,,
6,Zika Concerns are Spreading in Houston,United States,Houston,US,,
8,The CDC in Atlanta is Growing Worried,United States,Atlanta,US,,
10,Brownsville teen contracts Zika virus,United States,Brownsville,US,,
11,Mosquito control efforts in St. Louis take new...,United States,St. Louis,US,,


In [89]:
def getlatlong(ind):
    cd   = df['country_codes'][ind]
    city = df['cities'][ind]
    for items in gc.get_cities_by_name(city):
        for key in items.keys():
            if items[key]['countrycode'] == cd:
                df['lat'][ind] = items[key]['latitude']
                df['long'][ind] = items[key]['longitude']
        
    
for ind in df.index:    
    getlatlong(ind) 
    
df.head()

,headline,countries,cities,country_codes,lat,long
0,Zika Outbreak Hits Miami,United States,Miami,US,25.7743,-80.1937
1,Could Zika Reach New York City,United States,New York City,US,40.7143,-74.006
2,First Case of Zika in Miami Beach,United States,Miami Beach,US,25.7906,-80.13
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife,BR,-8.05389,-34.8811
4,Dallas man comes down with case of Zika,United States,Dallas,US,44.9193,-123.317


In [90]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 585 entries, 0 to 649
Data columns (total 6 columns):
headline         585 non-null object
countries        585 non-null object
cities           585 non-null object
country_codes    585 non-null object
lat              585 non-null object
long             585 non-null object
dtypes: object(6)
memory usage: 52.0+ KB


In [94]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='output.html')
